## Работает в среде Colab
https://drive.google.com/file/d/1-v_-XgfQLemy1K0uFJH8ma3W92cSKmW8/view?usp=sharing

## Загрузка Nemo


In [2]:
#Установка библиотеки 
## Установка зависимостей и самого немо
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2
!pip install --upgrade numba


## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]



  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=034d74d4c6d2cc4f6ce0b21573fc30529506ef467927043e3ced83f3e53b80b6
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-

In [1]:
#либы для саппорта

# import torch
import json
import glob
import os
import subprocess
import tarfile
import wget
import librosa
from tqdm.notebook import tqdm
import IPython
import pandas as pd

In [2]:
#Импорт библиотеки и колллекции моделей
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2022-02-09 19:24:14 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Загрузка датасета An4

In [ ]:
#Загрузка датасета An4 - небольшой английский, древний, очень простой датасет аудиофайлов с транскрибацией для тренировки и теста


# Текущая директория:
data_dir = '.'

#Загрузка датасета
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'http://www.speech.cs.cmu.edu/databases/an4/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'

if not os.path.exists(data_dir + '/an4/'):
    #распаковка архива и смена расширени при помощи библотеки sox и subprocess
    tar = tarfile.open(an4_path)
    tar.extractall(path=data_dir)

    print("Converting .sph to .wav...")
    sph_list = glob.glob(data_dir + '/an4/**/*.sph', recursive=True)
    for sph_path in sph_list:
        wav_path = sph_path[:-4] + '.wav'
        cmd = ["sox", sph_path, wav_path]
        subprocess.run(cmd)
print("Finished conversion.\n******")

Dataset downloaded at: ./an4_sphere.tar.gz
Converting .sph to .wav...
Finished conversion.
******


In [ ]:
#супер простой аудио файл на английском
audio_eng_file = ['./an4/wav/an4_clstk/mgah/cen2-mgah-b.wav']

#фраза проговоренная четко на английском
IPython.display.Audio(audio_eng_file[0])

In [ ]:
# Запись из репозитория 

audio_rus_file = ['/content/rus_text_audit_eto.mp3']

IPython.display.Audio(audio_rus_file[0])

##Импорт моделей

In [ ]:
# базовая модель
en_quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-01-26 16:01:51 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-01-26 16:01:52 common:729] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-26 16:01:53 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-01-26 16:01:53 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-01-26 16:01:53 features:264] PADDING: 16
[NeMo I 2022-01-26 16:01:53 features:281] STFT using torch
[NeMo I 2022-01-26 16:01:54 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-01-26 16:02:03 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [ ]:
#импорт лучшей предобученной модели на английском языке
stt_en_contextnet_1024= nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_en_contextnet_1024")

[NeMo I 2022-01-26 16:04:43 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_contextnet_1024/versions/1.6.0/files/stt_en_contextnet_1024.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_contextnet_1024/ad5749479c87c9e9c4ab3ebe491ebd99/stt_en_contextnet_1024.nemo
[NeMo I 2022-01-26 16:04:56 common:729] Instantiating model from pre-trained checkpoint
[NeMo I 2022-01-26 16:05:03 mixins:148] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-01-26 16:05:04 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    use_start_end_token: false
    
[NeMo W 2022-01-26 16:05:04 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-01-26 16:05:04 modelPT:149] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2022-01-26 16:05:04 features:264] PADDING: 16
[NeMo I 2022-01-26 16:05:04 features:281] STFT using torch


[NeMo W 2022-01-26 16:05:07 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      "num_layers={}".format(dropout, num_layers))
    
[NeMo W 2022-01-26 16:05:07 rnnt:741] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-01-26 16:05:07 rnnt_models:204] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001}
[NeMo I 2022-01-26 16:05:07 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-01-26 16:05:08 save_restore_connector:154] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_contextnet_1024/ad5749479c87c9e9c4ab3ebe491ebd99/stt_en_contextnet_1024.nemo.


In [ ]:
#импорт предобученной модели на русском язые, которая дана в репозитории nemo

rus_quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name='stt_ru_quartznet15x5')

[NeMo I 2022-01-22 21:13:58 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_ru_quartznet15x5/versions/1.0.0rc1/files/stt_ru_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0rc0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-01-22 21:14:01 common:729] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-22 21:14:02 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-01-22 21:14:02 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation da

[NeMo I 2022-01-22 21:14:02 features:264] PADDING: 16
[NeMo I 2022-01-22 21:14:02 features:281] STFT using torch
[NeMo I 2022-01-22 21:14:03 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-01-22 21:14:13 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0rc0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.


In [ ]:
# импорт лучшей asr conformer модели в коллекции предобученных образцов NeMo
stt_en_conformer_transducer_large = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_en_conformer_transducer_large")

[NeMo I 2022-01-26 16:08:57 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_transducer_large/versions/1.4.0/files/stt_en_conformer_transducer_large.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_conformer_transducer_large/3b5bf31cb5c6191963ed3c5b79ea5830/stt_en_conformer_transducer_large.nemo
[NeMo I 2022-01-26 16:09:09 common:729] Instantiating model from pre-trained checkpoint
[NeMo I 2022-01-26 16:09:15 mixins:148] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-01-26 16:09:16 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/nemo_asr_set/asr_set_1.4/train_no_appen/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: true
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/nemo_asr_set/asr_set_1.4/train_no_appen/audio__OP_0..2047_CL_.tar
    
[NeMo W 2022-01-26 16:09:16 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/libr

[NeMo I 2022-01-26 16:09:16 features:264] PADDING: 0
[NeMo I 2022-01-26 16:09:16 features:281] STFT using torch


[NeMo W 2022-01-26 16:09:17 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      "num_layers={}".format(dropout, num_layers))
    
[NeMo W 2022-01-26 16:09:17 rnnt:741] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-01-26 16:09:17 rnnt_models:204] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0}
[NeMo I 2022-01-26 16:09:17 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-01-26 16:09:18 save_restore_connector:154] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_conformer_transducer_large/3b5bf31cb5c6191963ed3c5b79ea5830/stt_en_conformer_transducer_large.nemo.


In [ ]:
#импорт конформера резульаты которого слегка хуже, чем тот, что импортирован выше. Этот работает на CTC лоссе 
stt_en_conformer_ctc_large  = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")

[NeMo I 2022-01-26 16:06:35 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large/versions/1.0.0/files/stt_en_conformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_conformer_ctc_large/b2da9ca277cfb30bb0c6b2abd44eef02/stt_en_conformer_ctc_large.nemo
[NeMo I 2022-01-26 16:06:48 common:729] Instantiating model from pre-trained checkpoint
[NeMo I 2022-01-26 16:06:52 mixins:148] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-01-26 16:06:52 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/nemo_asr_set/asr_set_1.4/train_no_appen/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/nemo_asr_set/asr_set_1.4/train_no_appen/audio__OP_0..2047_CL_.tar
    
[NeMo W 2022-01-26 16:06:52 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/l

[NeMo I 2022-01-26 16:06:52 features:264] PADDING: 0
[NeMo I 2022-01-26 16:06:52 features:281] STFT using torch
[NeMo I 2022-01-26 16:06:53 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-01-26 16:06:54 save_restore_connector:154] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_conformer_ctc_large/b2da9ca277cfb30bb0c6b2abd44eef02/stt_en_conformer_ctc_large.nemo.


## Транскрибация на примере разных моделей

In [ ]:
#проверка на простом тесте 
#вызов способа транскрибации 
stt_en_contextnet_1024.transcribe(paths2audio_files=audio_eng_file)[0]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['g l e n n']

In [ ]:
#как лучшая английская модель справится с русским аудио 
#текст русского: "Внутренний аудит это форма контроля деятельности организации изнутри. Процедура помогает руководству проверять финансовое состояние бизнеса и достоверность отчетности"
stt_en_contextnet_1024.transcribe(paths2audio_files=audio_rus_file)

#очень напоминает как если бы писали русские слова английской транскрипцией 

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-01-19 19:40:27 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


(['vunutrini oudid at a former controller dating estraganizadse is notream parasadura pomagad rocavosto previous financier by a custiani business it has diverse acting'],
 ['vunutrini oudid at a former controller dating estraganizadse is notream parasadura pomagad rocavosto previous financier by a custiani business it has diverse acting'])

In [ ]:
#проверка работы лучшего конформера на распознование русской речи 
#по какой-то причине выдаёт только часть предложения 

stt_en_conformer_transducer_large.transcribe(paths2audio_files=audio_rus_file)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-01-19 19:51:28 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


(['a form of controlling isi is notrencern'],
 ['a form of controlling isi is notrencern'])

In [ ]:
#акже напоминает транскрипцию русского написнную, английскими буквами, но в отличии от модели с transducer не отбрасывает большую часть слов 
stt_en_conformer_ctc_large.transcribe(paths2audio_files=audio_rus_file)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-01-19 19:56:56 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


['vutrni auditd a the former control aietister genizatse is nutri parazadura magatrukawosto prevared finanz stani business in st chotnsti']

In [ ]:
# и наконец русская модель, которая являет собой просто дообучение QuartzNet (обученного на 3к часах английских данных), с измененным слоем декодера куда вставлен русский алфавит
rus_quartznet.transcribe(paths2audio_files=audio_rus_file)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-01-19 20:05:35 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


['внутренний алдид это форума контроля деятельности организации изнутри процедуры помогает руководство проверять финансовое состояние бизнеса и достоверность отчетности']

## Построение манифеста для AN4

In [ ]:
#Построение манифеста(декларации аудио файлов)- это json файл, в котором в определенном порядке хранятся с метаданными о аудиозаписях

# Пример того как должна выглядить одна линия в данной декларации:
# {"audio_filepath": "path/to/audio.wav", "duration": 3.45, "text": "this is a nemo tutorial"}



# !!!!!!!!!!!!!!!!!!!!!!ДОПИСТАЬ
# функция для построения манифеста и добавление формата wav, спецом для работы с датасетом AN4 
# (в файлах an4 аудиофайлы представлены в формате sph (типичный формат для аудиофайлов, которые предполагается использоваться в сфере распознавания голоса)) 
def build_manifest(transcripts_path, manifest_path, wav_path):
    with open(transcripts_path, 'r') as fin:
        with open(manifest_path, 'w') as fout:
            for line in fin:
                # Lines look like this:
                # <s> transcript </s> (fileID)
                transcript = line[: line.find('(')-1].lower()
                transcript = transcript.replace('<s>', '').replace('</s>', '')
                transcript = transcript.strip()

                file_id = line[line.find('(')+1 : -2]  # e.g. "cen4-fash-b"
                audio_path = os.path.join(
                    data_dir, wav_path,
                    file_id[file_id.find('-')+1 : file_id.rfind('-')],
                    file_id + '.wav')

                duration = librosa.core.get_duration(filename=audio_path)

                # Write the metadata to the manifest
                metadata = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": transcript
                }
                json.dump(metadata, fout)
                fout.write('\n')

In [ ]:
#Создание манифеста для данных для трэина
train_transcripts = data_dir + '/an4/etc/an4_train.transcription'
train_manifest = data_dir + '/an4/train_manifest.json'

build_manifest(train_transcripts, train_manifest, 'an4/wav/an4_clstk')

In [ ]:
#манифест для теста
test_transcripts = data_dir + '/an4/etc/an4_test.transcription'
test_manifest = data_dir + '/an4/test_manifest.json'

build_manifest(test_transcripts, test_manifest, 'an4/wav/an4test_clstk')

## WER (An4) eng Quartznet 15х5
0.077



In [ ]:
#добавление манифеста в quartznet 

en_quartznet._cfg['validation_ds']['manifest_filepath'] = test_manifest
en_quartznet._cfg['validation_ds']


{'manifest_filepath': './an4/test_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}

In [ ]:
# Quartznet

en_quartznet._cfg['validation_ds']['batch_size'] = 5

# Настройка загрузчика тестовых данных и убедитесь, что модель находится на графическом процессоре.
en_quartznet.setup_test_data(test_data_config=en_quartznet._cfg['validation_ds'])
en_quartznet.cuda()
en_quartznet.eval()

# WER is computed as numerator/denominator. (числитель/знаменатель)
# Соберем все числители и знаменатели тестовых партий
wer_nums = []
wer_denoms = []


for test_batch in en_quartznet.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    
        #обычно снчала пердаётся входящий сигнал, потом длинна батча    
# ``input_signal`` and ``input_signal_length`` are mutually exclusive  with ``processed_signal`` and ``processed_signal_len`` arguments.

        log_probs, encoded_len, greedy_predictions = en_quartznet(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        
        # print('предсказания: ', greedy_predictions)
        # print('энкодированная длина:', encoded_len)


        # Notice the model has a helper object to compute WER
        en_quartznet._wer.update(greedy_predictions, targets, targets_lengths)

        _, wer_num, wer_denom= en_quartznet._wer.compute()

        en_quartznet._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, targets, targets_lengths, encoded_len, greedy_predictions


# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-01-26 16:03:59 collections:173] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2022-01-26 16:03:59 collections:174] 0 files were filtered totalling 0.00 hours
WER = 0.07761966364812418


## WER eng_contextnet

0.0362

In [ ]:
#передаем внутрь конфига модели информацию о манифесте тренировочного датасета, чтобы в дальнейшем к ней обращаться при рассчёте WER
stt_en_contextnet_1024._cfg['validation_ds']['manifest_filepath'] = test_manifest

#проверка, что действительно что-то появилось в моделе контексте_1024
stt_en_contextnet_1024._cfg['validation_ds']
#пояилась надписть после 'manifest_filepath':

{'manifest_filepath': './an4/test_manifest.json', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'use_start_end_token': False}

In [ ]:
#stt_en_contextnet_1024

stt_en_contextnet_1024._cfg['validation_ds']['batch_size'] = 4

# Настройка загрузчика тестовых данных и убедитесь, что модель находится на графическом процессоре.
stt_en_contextnet_1024.setup_test_data(test_data_config=stt_en_contextnet_1024._cfg['validation_ds'])
stt_en_contextnet_1024.cuda()
stt_en_contextnet_1024.eval()

wer_nums = []
wer_denoms = []


for test_batch in stt_en_contextnet_1024.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    


        greedy_predictions,  encoded_len = stt_en_contextnet_1024(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        
 
        stt_en_contextnet_1024.wer.update(greedy_predictions, encoded_len, targets, targets_lengths)

        _, wer_num, wer_denom= stt_en_contextnet_1024.wer.compute()

        stt_en_contextnet_1024.wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, targets, targets_lengths, encoded_len, greedy_predictions


# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-01-26 16:05:39 collections:173] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2022-01-26 16:05:39 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-01-26 16:05:39 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:05:39 rnnt_wer_bpe:232] reference :rubout g m e f three nine
[NeMo I 2022-01-26 16:05:39 rnnt_wer_bpe:233] predicted :rubout g m e f three nine
[NeMo I 2022-01-26 16:05:39 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:05:39 rnnt_wer_bpe:232] reference :rubout n i m n one
[NeMo I 2022-01-26 16:05:39 rnnt_wer_bpe:233] predicted :rubout n i m n one
[NeMo I 2022-01-26 16:05:40 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:05:40 rnnt_wer_bpe:232] reference :m e l v i n
[NeMo I 2022-01-26 16:05:40 rnnt_wer_bpe:233] predicted :m e l v i n
[NeMo I 2022-01-26 16:05:40 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:05:40 rnnt_wer_bpe:232] reference :eleven twenty seven fifty seven
[NeMo I 2022-01-26 16:05:40 rnnt_wer_bpe:233] predicted :eleven

## WER eng Conformer-CTC

0.0245

In [ ]:
#доавбление внутрь конформера данных о тестовом манифесте 
stt_en_conformer_ctc_large._cfg['validation_ds']['manifest_filepath'] = test_manifest

#проверка, что появилось в конформере_стс_лардж
stt_en_conformer_ctc_large._cfg['validation_ds']

{'manifest_filepath': './an4/test_manifest.json', 'sample_rate': 16000, 'batch_size': 8, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'is_tarred': False, 'tarred_audio_filepaths': ''}

In [ ]:
#WER для Конформера CTC loss

stt_en_conformer_ctc_large._cfg['validation_ds']['batch_size'] = 4

# Настройка загрузчика тестовых данных и убедитесь, что модель находится на графическом процессоре.
stt_en_conformer_ctc_large.setup_test_data(test_data_config=stt_en_conformer_ctc_large._cfg['validation_ds'])
stt_en_conformer_ctc_large.cuda()
stt_en_conformer_ctc_large.eval()

wer_nums = []
wer_denoms = []


for test_batch in stt_en_conformer_ctc_large.test_dataloader():
        print('='*20)
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    


        log_probs, encoded_len, greedy_predictions = stt_en_conformer_ctc_large(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        

        stt_en_conformer_ctc_large._wer.update(greedy_predictions, targets, target_lengths=targets_lengths)
        _, wer_num, wer_denom= stt_en_conformer_ctc_large._wer.compute()

        stt_en_conformer_ctc_large._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        del test_batch, targets, targets_lengths, greedy_predictions
        print('='*20)

print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-01-26 16:08:24 collections:173] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2022-01-26 16:08:24 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-01-26 16:08:24 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-01-26 16:08:25 wer_bpe:203] 
    
[NeMo I 2022-01-26 16:08:25 wer_bpe:204] reference:rubout g m e f three nine
[NeMo I 2022-01-26 16:08:25 wer_bpe:205] predicted:rubbout g m e f three nine
[NeMo I 2022-01-26 16:08:25 wer_bpe:203] 
    
[NeMo I 2022-01-26 16:08:25 wer_bpe:204] reference:rubout n i m n one
[NeMo I 2022-01-26 16:08:25 wer_bpe:205] predicted:rubout n i m n one
[NeMo I 2022-01-26 16:08:25 wer_bpe:203] 
    
[NeMo I 2022-01-26 16:08:25 wer_bpe:204] reference:m e l v i n
[NeMo I 2022-01-26 16:08:25 wer_bpe:205] predicted:m e l v i n
[NeMo I 2022-01-26 16:08:25 wer_bpe:203] 
    
[NeMo I 2022-01-26 16:08:25 wer_bpe:204] reference:eleven twenty seven fifty seven
[NeMo I 2022-01-26 16:08:25 wer_bpe:205] predicted:eleven twenty seven fifty seven
[NeMo I 2022-01-26 16:08:25 wer_bpe:203] 
    
[NeMo I 2022-01-26 16:08:25 wer_bpe:204] reference:j p e g four
[NeMo I 2022-01-26 16:08:25 wer_bpe:205] predicted:j p e g four
[NeMo I 2022-01-26 16:08:25 wer_bpe:203] 
    
[Ne

## WER Conformer-Transducer

0.0724

In [ ]:
#определение тренировочной выборки в модель
stt_en_conformer_transducer_large._cfg['validation_ds']['manifest_filepath'] = test_manifest
stt_en_conformer_transducer_large._cfg['validation_ds']

{'manifest_filepath': './an4/test_manifest.json', 'sample_rate': 16000, 'batch_size': 8, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': True, 'is_tarred': False, 'tarred_audio_filepaths': '/data/LibriSpeech/eval__OP_0..1023_CL_.tar'}

In [ ]:
stt_en_conformer_transducer_large._cfg['validation_ds']['batch_size'] = 4


stt_en_conformer_transducer_large.setup_test_data(test_data_config=stt_en_conformer_transducer_large._cfg['validation_ds'])
stt_en_conformer_transducer_large.cuda()
stt_en_conformer_transducer_large.eval()

# Соберем все числители и знаменатели тестовых партий
wer_nums = []
wer_denoms = []


for test_batch in stt_en_conformer_transducer_large.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    
      

        greedy_predictions, encoded_len = stt_en_conformer_transducer_large(
            input_signal=test_batch[0], input_signal_length=test_batch[1])


        stt_en_conformer_transducer_large.wer.update(greedy_predictions, encoded_len, targets, targets_lengths)
        _, wer_num, wer_denom= stt_en_conformer_transducer_large.wer.compute()

        stt_en_conformer_transducer_large.wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, targets, targets_lengths, greedy_predictions

# # We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-01-26 16:10:24 collections:173] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2022-01-26 16:10:24 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-01-26 16:10:24 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-01-26 16:10:24 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:10:24 rnnt_wer_bpe:232] reference :rubout g m e f three nine
[NeMo I 2022-01-26 16:10:24 rnnt_wer_bpe:233] predicted :rebel g m e f three nine
[NeMo I 2022-01-26 16:10:24 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:10:24 rnnt_wer_bpe:232] reference :rubout n i m n one
[NeMo I 2022-01-26 16:10:24 rnnt_wer_bpe:233] predicted :rub out n i m n one
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:232] reference :m e l v i n
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:233] predicted :me l v i n
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:232] reference :eleven twenty seven fifty seven
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:233] predicted :eleven twenty seven fifty seven
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:231] 
    
[NeMo I 2022-01-26 16:10:25 rnnt_wer_bpe:232] reference :j p e g four
[NeMo I 2022-01-26 16:10:25 rnnt_wer_

## Загрузка и работа с датасетом MCV rus

In [ ]:
# Просмотр активированной колабовской конфигурации
!nvidia-smi

Wed Jan 26 16:12:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    32W / 250W |   7573MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#для удаления папок
!rm -R /content/datasets


rm: cannot remove '/content/datasets': No such file or directory


In [ ]:
!sudo apt-get install sox libsox-fmt-mp3


Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following additional packages will be installed:
  libid3tag0 libmad0
The following NEW packages will be installed:
  libid3tag0 libmad0 libsox-fmt-mp3
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 112 kB of archives.
After this operation, 370 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libid3tag0 amd64 0.15.1b-13 [31.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libmad0 amd64 0.15.1b-9ubuntu18.04.1 [64.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libsox-fmt-mp3 amd64 14.4.2-3ubuntu0.18.04.1 [15.9 kB]
Fetched 112 kB in 0s (352 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /us

In [ ]:
#подготовленный пайтон код для работы с датасетом mcv

!wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/get_commonvoice_data.py

--2022-02-02 07:54:47--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/get_commonvoice_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6778 (6.6K) [text/plain]
Saving to: ‘scripts/get_commonvoice_data.py’

get_commonvoice_dat 100%[===================>]   6.62K  --.-KB/s    in 0s      

2022-02-02 07:54:48 (84.7 MB/s) - ‘scripts/get_commonvoice_data.py’ saved [6778/6778]



In [ ]:
VERSION = "cv-corpus-6.1-2020-12-11"
LANGUAGE = "ru"

tokenizer_dir = os.path.join('tokenizers', LANGUAGE)
manifest_dir = os.path.join('manifests', LANGUAGE)

In [ ]:
#папка куда будем складывать датасеты 
!mkdir -p datasets

In [ ]:
#русскому датасету скачиваться и разархивироваться и примерно 20 минут, validated.tsv загружается уде 36 минут один, так что его лучше не грузить, если нет цели его использовать
#общее время скачивания 'train.tsv' 'dev.tsv' 'test.tsv' 'validated.tsv'  ~50 минут

!python scripts/get_commonvoice_data.py \
  --data_root "datasets/$LANGUAGE/" \
  --manifest_dir=$manifest_dir \
  --sample_rate=16000 \
  --n_channels=1 \
  --version=$VERSION \
  --language=$LANGUAGE \
  --files_to_process 'train.tsv' 'dev.tsv' 'test.tsv'

  #--files_to_process 'train.tsv' 'dev.tsv' 'test.tsv' 'validated.tsv'

--2022-02-02 07:54:48--  https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/ru.tar.gz
Resolving voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)... 52.92.146.33
Connecting to voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)|52.92.146.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3655676916 (3.4G) [application/octet-stream]
Saving to: ‘datasets/ru/ru.tar.gz’

ru.tar.gz           100%[===================>]   3.40G  12.9MB/s    in 3m 21s  

2022-02-02 07:58:09 (17.4 MB/s) - ‘datasets/ru/ru.tar.gz’ saved [3655676916/3655676916]

100% 15480/15480 [06:56<00:00, 37.18it/s]
100% 15480/15480 [00:00<00:00, 77012.10it/s]
100% 7962/7962 [03:45<00:00, 35.32it/s]
100% 7962/7962 [00:00<00:00, 86022.93i

In [ ]:
train_manifest = f"{manifest_dir}/commonvoice_train_manifest.json"
dev_manifest = f"{manifest_dir}/commonvoice_dev_manifest.json"
test_manifest = f"{manifest_dir}/commonvoice_test_manifest.json"
# val_manifest = f"{manifest_dir}/commonvoice_validated_manifest.json"

In [ ]:
dev_manifest

'manifests/ru/commonvoice_dev_manifest.json'

## Тест QuartzNetа из репозитория
заявленная WER на dev части датасета MCV 16,3% 
после проверки получилось только 17,2%

?Скорее всего связано с методикой подсчёта WER?

In [ ]:
rus_quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name='stt_ru_quartznet15x5')

[NeMo I 2022-01-25 06:57:22 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_ru_quartznet15x5/versions/1.0.0rc1/files/stt_ru_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-01-25 06:57:26 common:729] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-25 06:57:27 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-01-25 06:57:27 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation da

[NeMo I 2022-01-25 06:57:27 features:264] PADDING: 16
[NeMo I 2022-01-25 06:57:27 features:281] STFT using torch
[NeMo I 2022-01-25 06:57:28 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-01-25 06:57:37 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.


In [ ]:

rus_quartznet._cfg['train_ds']

{'manifest_filepath': '/raid/noneval.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'num_workers': 8, 'pin_memory': True}

In [ ]:
rus_quartznet._cfg['validation_ds']

{'manifest_filepath': '/raid/dev.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 32, 'shuffle': False, 'num_workers': 8, 'pin_memory': True}

In [ ]:
rus_quartznet.cfg.keys()

dict_keys(['preprocessor', 'spec_augment', 'encoder', 'decoder', 'optim', 'target', 'train_ds', 'validation_ds'])

In [ ]:
rus_quartznet._cfg['validation_ds']['manifest_filepath'] = dev_manifest

In [ ]:
rus_quartznet._cfg

{'preprocessor': {'_target_': 'nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor', 'normalize': 'per_feature', 'window_size': 0.02, 'sample_rate': 16000, 'window_stride': 0.01, 'window': 'hann', 'features': 64, 'n_fft': 512, 'frame_splicing': 1, 'dither': 1e-05, 'stft_conv': False}, 'spec_augment': {'_target_': 'nemo.collections.asr.modules.SpectrogramAugmentation', 'rect_freq': 50, 'rect_masks': 5, 'rect_time': 120}, 'encoder': {'_target_': 'nemo.collections.asr.modules.ConvASREncoder', 'feat_in': 64, 'activation': 'relu', 'conv_mask': True, 'jasper': [{'filters': 256, 'repeat': 1, 'kernel': [33], 'stride': [2], 'dilation': [1], 'dropout': 0.0, 'residual': False, 'separable': True}, {'filters': 256, 'repeat': 5, 'kernel': [33], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True}, {'filters': 256, 'repeat': 5, 'kernel': [33], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True}, {'filters': 256, 'repeat': 5, 

In [ ]:
#чтобы не появлялись логи во время обучения, результаты работы модели
rus_quartznet._wer.log_prediction = True

In [ ]:
rus_quartznet._wer.log_prediction

True

In [ ]:
rus_quartznet._cfg['validation_ds']['batch_size'] = 10 #чем больше данное значение, тем больше пропускная способность, тем быстрее посчитается
# Настройка загрузчика тестовых данных и убедитесь, что модель находится на графическом процессоре.
rus_quartznet.setup_test_data(test_data_config=rus_quartznet.cfg['validation_ds'])
rus_quartznet.cuda()
rus_quartznet.eval()
# WER is computed as numerator/denominator. (числитель/знаменатель)
# Соберем все числители и знаменатели тестовых партий
wer_nums = []
wer_denoms = []


for test_batch in tqdm(rus_quartznet.test_dataloader()):
        # print('='*20)
        # test_batch = [x.cuda() for x in test_batch]
        test_batch = [x.cuda() for x in test_batch]

        targets = test_batch[2]
        targets_lengths = test_batch[3]    


        log_probs, encoded_len, greedy_predictions = rus_quartznet(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        
   
        rus_quartznet._wer.update(predictions=greedy_predictions, 
                                  targets=targets, 
                                  target_lengths=targets_lengths, 
                                  predictions_lengths=encoded_len)


        wer, wer_num, wer_denom= rus_quartznet._wer.compute()

        # print(float(wer_num)/ float(wer_denom))
        # print(wer)

        rus_quartznet._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        
        del test_batch, targets, targets_lengths, encoded_len, greedy_predictions, log_probs
        # print('='*20)


# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-01-25 06:59:18 audio_to_text_dataset:42] Model level config does not container `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2022-01-25 06:59:18 audio_to_text_dataset:42] Model level config does not container `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2022-01-25 06:59:18 collections:173] Dataset loaded with 7962 files totalling 12.36 hours
[NeMo I 2022-01-25 06:59:18 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-01-25 06:59:18 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


  0%|          | 0/797 [00:00<?, ?it/s]

[NeMo I 2022-01-25 06:59:20 wer:244] 
    
[NeMo I 2022-01-25 06:59:20 wer:245] reference:их осуществление видимо будет сложным и потребует времени
[NeMo I 2022-01-25 06:59:20 wer:246] predicted:их осуществлении видимо будет сложным и потребует времени
[NeMo I 2022-01-25 06:59:20 wer:244] 
    
[NeMo I 2022-01-25 06:59:20 wer:245] reference:нам надо защищать и поощрять права человека как фундамент политической стабильности и устойчивого роста
[NeMo I 2022-01-25 06:59:20 wer:246] predicted:нам надо защищать и поощрять права человека как фундамент политической стабильности и устойчивого роста
[NeMo I 2022-01-25 06:59:20 wer:244] 
    
[NeMo I 2022-01-25 06:59:20 wer:245] reference:какие действия предпринял совет безопасности в отношении этих ужасных расправ которые безнаказанно продолжаются
[NeMo I 2022-01-25 06:59:20 wer:246] predicted:какие действия предпринял совет безопасности в отношении этих ужасных расправ которые безнаказанно продолжается
[NeMo I 2022-01-25 06:59:20 wer:244] 
   

In [ ]:
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

WER = 0.15085754287714387


In [ ]:
print('WER2:', sum())

## Изменим и дообучим модель QuartzNet на русских данных

In [ ]:
#Сами попробуем дообучить quartznet на русском языке, но большего размера. Цель: получить результат wer меньше чем 17,2% (результат у стоковой русской модели от nvidia)
#Модель старшего поколения предобученная на большем кол-ве данных чем там что использовалась для создания русского quartznet

In [ ]:
!nvidia-smi

Tue Jan 25 19:27:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#данная модель оубчалась на датасетах общей продолжительностью 7к часов, в то время как тот quartznet который был использован для переобучения на русском языке был обучен на 3к
rusfromeng_quartz= nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-01-25 19:28:12 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-01-25 19:28:15 common:729] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-25 19:28:16 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-01-25 19:28:16 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-01-25 19:28:16 features:264] PADDING: 16
[NeMo I 2022-01-25 19:28:16 features:281] STFT using torch
[NeMo I 2022-01-25 19:28:17 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-01-25 19:28:27 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.6.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [ ]:
#Взаимодействие с моделями от немо всегда происходит через их кофиг файл
# посмотреть какие параметры есть внутри архитектуры можно командой вызова ключей 
rusfromeng_quartz.cfg.keys()

dict_keys(['sample_rate', 'repeat', 'dropout', 'separable', 'labels', 'train_ds', 'validation_ds', 'preprocessor', 'spec_augment', 'encoder', 'decoder', 'optim', 'target'])

In [ ]:
rusfromeng_quartz.cfg.optim.sched

{'name': 'CosineAnnealing', 'warmup_steps': None, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}

In [ ]:
#новый словарь 
new_vocab = [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']

In [ ]:
#замена словаря на русские буквы 
rusfromeng_quartz.change_vocabulary(new_vocab)

[NeMo I 2022-01-25 07:10:05 ctc_models:365] Changed decoder to output to [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'] vocabulary.


In [ ]:
rusfromeng_quartz.cfg.decoder['num_classes']

34

In [ ]:
rusfromeng_quartz.cfg.labels = new_vocab

In [ ]:
# проверка
rusfromeng_quartz.cfg.decoder.vocabulary

[' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']

In [ ]:
#нужно если хотим обучать только декодер, не трогая основные веса модели
#заморозка слоёв энкодера, но оставляем батч нормализацию 

import torch
import torch.nn as nn

def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

rusfromeng_quartz.encoder.freeze()
rusfromeng_quartz.encoder.apply(enable_bn_se)

ConvASREncoder(
  (encoder): Sequential(
    (0): JasperBlock(
      (mconv): ModuleList(
        (0): MaskedConv1d(
          (conv): Conv1d(64, 64, kernel_size=(33,), stride=(2,), padding=(16,), groups=64, bias=False)
        )
        (1): MaskedConv1d(
          (conv): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
        )
        (2): BatchNorm1d(256, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (mout): Sequential(
        (0): ReLU(inplace=True)
        (1): Dropout(p=0.0, inplace=False)
      )
    )
    (1): JasperBlock(
      (mconv): ModuleList(
        (0): MaskedConv1d(
          (conv): Conv1d(256, 256, kernel_size=(33,), stride=(1,), padding=(16,), groups=256, bias=False)
        )
        (1): MaskedConv1d(
          (conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
        )
        (2): BatchNorm1d(256, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
      

In [ ]:
rusfromeng_quartz.cfg.encoder

{'_target_': 'nemo.collections.asr.modules.ConvASREncoder', 'feat_in': 64, 'activation': 'relu', 'conv_mask': True, 'jasper': [{'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [33], 'repeat': 1, 'residual': False, 'separable': True, 'stride': [2]}, {'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [33], 'repeat': 5, 'residual': True, 'separable': True, 'stride': [1]}, {'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [33], 'repeat': 5, 'residual': True, 'separable': True, 'stride': [1]}, {'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [33], 'repeat': 5, 'residual': True, 'separable': True, 'stride': [1]}, {'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [39], 'repeat': 5, 'residual': True, 'separable': True, 'stride': [1]}, {'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [39], 'repeat': 5, 'residual': True, 'separable': True, 'stride': [1]}, {'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [39], 'repeat': 5, '

In [ ]:
rusfromeng_quartz.cfg.decoder

{'_target_': 'nemo.collections.asr.modules.ConvASRDecoder', 'feat_in': 1024, 'num_classes': 34, 'vocabulary': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']}

In [ ]:

rusfromeng_quartz.cfg

{'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'train_ds': {'manifest_filepath': '/data2/voices/train_1k.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': '/asr_set_1.2/train/train_{0..1023}.tar', 'num_workers': 20}, 'validation_ds': {'manifest_filepath': '/data2/voices/train_1k_samp.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 32, 'shuffle': False}, 'prep

In [ ]:

#определяем в конфиге модели тренировочный и тестовый манифест 
rusfromeng_quartz._cfg['train_ds']['manifest_filepath']  = train_manifest
rusfromeng_quartz._cfg['validation_ds']['manifest_filepath'] = test_manifest

NameError: ignored

In [ ]:
# ru_quartz._cfg['train_ds']['is_tarred'] = False
rusfromeng_quartz.setup_training_data(train_data_config=rusfromeng_quartz.cfg['train_ds'])
rusfromeng_quartz.setup_validation_data(val_data_config=rusfromeng_quartz._cfg['validation_ds'])

[NeMo I 2022-01-25 07:19:47 collections:173] Dataset loaded with 15480 files totalling 23.48 hours
[NeMo I 2022-01-25 07:19:47 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-01-25 07:19:47 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-01-25 07:19:47 collections:173] Dataset loaded with 8006 files totalling 13.24 hours
[NeMo I 2022-01-25 07:19:47 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
rusfromeng_quartz._cfg['validation_ds']['batch_size'] 

32

In [ ]:
#устанавливаем размеры батчей исходя из возможностей gpu на компьюетере 
rusfromeng_quartz._cfg['train_ds']['batch_size'] = 32     #значально 32 
rusfromeng_quartz._cfg['validation_ds']['batch_size'] = 32   # изначально 32

In [ ]:
#чтобы смотреть логи во время обучения, показывают референс и предикт
c

In [ ]:
rusfromeng_quartz.cfg

{'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'train_ds': {'manifest_filepath': 'manifests/ru/commonvoice_train_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': '/asr_set_1.2/train/train_{0..1023}.tar', 'num_workers': 20}, 'validation_ds': {'manifest_filepath': 'manifests/ru/commonvoice_test_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 

In [ ]:
# для замены показателей в оптимайезере
# rusfromeng_quartz.cfg.optim.lr = 0.01 

In [ ]:
#обучеие происходит через библиотеку pytorch_lightning
import pytorch_lightning as pl
#прописываем кол-во гпу и эпох 
trainer = pl.Trainer(gpus=1, max_epochs=150)

#Тренер с несколькими гиперпараметрами
# trainer = pl.Trainer(gpus=1,                        #кол-во используемых гпу
#                       max_epochs=3,                 #кол-во эпох 
#                       accumulate_grad_batches=1,    #accumulates grads every k batches or as set up in the dict
#                       checkpoint_callback=False,    #создавать или нет чекпоинты 
#                       logger=False,                 #покажет логи во время обучения 
#                       log_every_n_steps=5,          #как часто выполнять вход в пределах шагов (по умолчанию каждые 50 шагов). 
#                       check_val_every_n_epoch=10)   #проверять значение каждые столько-то эпох

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
# 10 минут на одну эпоху

# с 20 эпохи
%%time
trainer.fit(rusfromeng_quartz)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2022-01-25 15:58:46 modelPT:458] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-01-25 15:58:46 modelPT:566] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )


[NeMo W 2022-01-25 15:58:46 lr_scheduler:817] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | _wer              | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 35.9 K
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.722    Total estimated model params size (MB)
[NeMo W 2022-01-25 15:58:47 nemo_logging:349] /usr/loc

Validation sanity check: 0it [00:00, ?it/s]

[NeMo I 2022-01-25 15:58:47 wer:244] 
    
[NeMo I 2022-01-25 15:58:47 wer:245] reference:если не будет возражений я буду считать что ассамблея согласна с этим предложением
[NeMo I 2022-01-25 15:58:47 wer:246] predicted:если не будит возражений я буду считать чо ассаблеи и совасно ж этим предложения
[NeMo I 2022-01-25 15:58:47 wer:244] 
    
[NeMo I 2022-01-25 15:58:47 wer:245] reference:господин председатель мы рассчитываем прилежно поработать с вами над докладом
[NeMo I 2022-01-25 15:58:47 wer:246] predicted:господин председатель мы расчитаем по лежно поработать с вамин бондокладом


Training: 0it [00:00, ?it/s]

[NeMo W 2022-01-25 15:58:48 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-01-25 15:58:53 wer:244] 
    
[NeMo I 2022-01-25 15:58:53 wer:245] reference:я предоставляю слово господину линну пэскоу
[NeMo I 2022-01-25 15:58:53 wer:246] predicted:я предоставляю слово господину линну пэск
[NeMo I 2022-01-25 15:58:54 wer:244] 
    
[NeMo I 2022-01-25 15:58:54 wer:245] reference:все группы должны прислушаться к этому призыву
[NeMo I 2022-01-25 15:58:54 wer:246] predicted:все группы должны присувшаься к этому призыву
[NeMo I 2022-01-25 15:58:55 wer:244] 
    
[NeMo I 2022-01-25 15:58:55 wer:245] reference:есть ли какиенибудь замечания по этим просьбам
[NeMo I 2022-01-25 15:58:55 wer:246] predicted:естьли какиениудь замечания по этим просьбам
[NeMo I 2022-01-25 15:58:56 wer:244] 
    
[NeMo I 2022-01-25 15:58:56 wer:245] reference:пока продолжает существовать это оружие данный форум не может расслабляться или сохранять невозмутимость
[NeMo I 2022-01-25 15:58:56 wer:246] predicted:пока продолжает существовать это оружие данный форум не может расслабляться 

[NeMo W 2022-01-25 16:00:41 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    


CPU times: user 1min 34s, sys: 16.5 s, total: 1min 50s
Wall time: 1min 55s


In [ ]:


save_path = f"Rus_from_Eng_quartznet47_epoch-{LANGUAGE}.nemo"
rusfromeng_quartz.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")

Model saved at path : /content/Rus_from_Eng_quartznet47_epoch-ru.nemo


## Просто пример дообучения модели без добавления нового словаря, также можно дообучать на новых данных любую другую модель в которой нет токенизатора


В данном примере продолжается процесс дообучения модели, который был прерван выше

In [ ]:
#файл весов модели есть в репозитории

rus_from_eng = nemo_asr.models.EncDecCTCModel.restore_from("/content/Rus_from_Eng_quartznet253_epoch-ru.nemo")

[NeMo W 2022-02-01 09:59:23 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: manifests/ru/commonvoice_train_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-02-01 09:59:23 modelPT:150] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid confi

[NeMo I 2022-02-01 09:59:23 features:264] PADDING: 16
[NeMo I 2022-02-01 09:59:23 features:281] STFT using torch
[NeMo I 2022-02-01 09:59:24 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-02-01 09:59:33 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /content/Rus_from_Eng_quartznet253_epoch-ru.nemo.


In [ ]:
rus_from_eng.cfg.train_ds.batch_size, rus_from_eng.cfg.test_ds.batch_size, rus_from_eng.cfg.validation_ds.batch_size

(32, 16, 16)

In [ ]:
rus_from_eng.cfg.test_ds.batch_size = 16
rus_from_eng.cfg.validation_ds.batch_size = 16

In [ ]:
rus_from_eng._cfg['train_ds']['manifest_filepath']  = train_manifest
rus_from_eng._cfg['validation_ds']['manifest_filepath'] = test_manifest

In [ ]:
rus_from_eng._cfg['train_ds']

{'manifest_filepath': 'manifests/ru/commonvoice_train_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': '/asr_set_1.2/train/train_{0..1023}.tar', 'num_workers': 20}

In [ ]:
rus_from_eng._cfg['validation_ds']

{'manifest_filepath': 'manifests/ru/commonvoice_test_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'shuffle': False}

In [ ]:

rus_from_eng.setup_training_data(train_data_config=rus_from_eng.cfg['train_ds'])
rus_from_eng.setup_validation_data(val_data_config=rus_from_eng.cfg['validation_ds'])

[NeMo I 2022-02-01 09:59:34 collections:173] Dataset loaded with 15480 files totalling 23.48 hours
[NeMo I 2022-02-01 09:59:34 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-02-01 09:59:34 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-02-01 09:59:34 collections:173] Dataset loaded with 8006 files totalling 13.24 hours
[NeMo I 2022-02-01 09:59:34 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
import pytorch_lightning as pl

trainer = pl.Trainer(gpus=1, max_epochs=100)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
rus_from_eng._wer.log_prediction = True

In [ ]:
rus_from_eng.cfg.optim

{'name': 'novograd', 'lr': 0.0003, 'betas': [0.8, 0.5], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': None, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}

In [ ]:
#спустя ~90 эпох заметил что loss уперся в 11, повышение lr приводило к увеличению ошибок, уменьшение дало прирост качества (уменьшал пошагово от 0.006 до 0.0002)
rus_from_eng.cfg.optim.lr = 0.0002

In [ ]:
# %%time 
# с 17
trainer.fit(rus_from_eng)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2022-02-01 21:14:35 modelPT:465] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-02-01 21:14:35 modelPT:577] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0002
        weight_decay: 0.001
    )


[NeMo W 2022-02-01 21:14:35 lr_scheduler:817] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 35.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.722    Total estimated model params size (MB)
[NeMo W 2022-02-01 21:14:35 nemo_logging:349] /usr/loc

Validation sanity check: 0it [00:00, ?it/s]

[NeMo I 2022-02-01 21:14:36 wer:244] 
    
[NeMo I 2022-02-01 21:14:36 wer:245] reference:если не будет возражений я буду считать что ассамблея согласна с этим предложением
[NeMo I 2022-02-01 21:14:36 wer:246] predicted:если не буде возражени я будут считать что ассамблее согласны с этим предложением
[NeMo I 2022-02-01 21:14:36 wer:244] 
    
[NeMo I 2022-02-01 21:14:36 wer:245] reference:я не вру
[NeMo I 2022-02-01 21:14:36 wer:246] predicted:я не вру


Training: 0it [00:00, ?it/s]

[NeMo W 2022-02-01 21:14:36 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-02-01 21:14:43 wer:244] 
    
[NeMo I 2022-02-01 21:14:43 wer:245] reference:однако существующие запасы ядерного оружия попрежнему представляют собой серьезную угрозу для человечества
[NeMo I 2022-02-01 21:14:43 wer:246] predicted:однако существующие запасы ядерного оружия попрежнему представляют собой серьезную угрозу для человечества
[NeMo I 2022-02-01 21:14:44 wer:244] 
    
[NeMo I 2022-02-01 21:14:44 wer:245] reference:для удобства пользования мы распространили текст этих изменений среди всех делегаций в этом зале
[NeMo I 2022-02-01 21:14:44 wer:246] predicted:для удобства пользования мы распространили текст этих изменений среди всех делегаций в этом зале
[NeMo I 2022-02-01 21:14:45 wer:244] 
    
[NeMo I 2022-02-01 21:14:45 wer:245] reference:поэтому мы воздержались при голосовании
[NeMo I 2022-02-01 21:14:45 wer:246] predicted:поэтому мы воздержались при голосовании
[NeMo I 2022-02-01 21:14:47 wer:244] 
    
[NeMo I 2022-02-01 21:14:47 wer:245] reference:я безынициа

[NeMo W 2022-02-01 21:14:48 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    


In [ ]:
save_path = f"Rus_from_Eng_quartznet311_epoch-{LANGUAGE}.nemo"
rus_from_eng.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")

Model saved at path : /content/Rus_from_Eng_quartznet311_epoch-ru.nemo


In [ ]:
/content/Rus_from_Eng_quartznet47_epoch-ru.nemo

In [ ]:
rus_from_eng._cfg['validation_ds']['manifest_filepath'] = dev_manifest

#проверка, что появилось в конформере_стс_лардж
rus_from_eng._cfg['validation_ds']

{'manifest_filepath': 'manifests/ru/commonvoice_dev_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'shuffle': False}

In [ ]:
rus_from_eng._cfg['validation_ds']['batch_size'] = 8

#Настройка загрузчика тестовых данных
rus_from_eng.setup_test_data(test_data_config=rus_from_eng._cfg['validation_ds'])
rus_from_eng.cuda()
rus_from_eng.eval()

wer_nums = []
wer_denoms = []


for test_batch in rus_from_eng.test_dataloader():
        print('='*20)
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    

        log_probs, encoded_len, greedy_predictions = rus_from_eng(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        
        rus_from_eng._wer.update(greedy_predictions, targets, targets_lengths)

        wer, wer_num, wer_denom= rus_from_eng._wer.compute()
        print(wer)

        rus_from_eng._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, targets, targets_lengths, encoded_len, greedy_predictions
        print('='*20)


# делим сумму всех числителей и знаменателей
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

Streaming output truncated to the last 5000 lines.
[NeMo I 2022-02-01 21:15:47 wer:244] 
    
[NeMo I 2022-02-01 21:15:47 wer:245] reference:мы готовимся предпринять следующие шаги в области обеспечения контроля и разоружения
[NeMo I 2022-02-01 21:15:47 wer:246] predicted:мы готовимся предринять следующие шаги в области обеспечения контроля и разоружению
tensor(0.5802, device='cuda:0')
[NeMo I 2022-02-01 21:15:48 wer:244] 
    
[NeMo I 2022-02-01 21:15:48 wer:245] reference:что делают соединенные штаты
[NeMo I 2022-02-01 21:15:48 wer:246] predicted:что делают соединенные штаты
tensor(0.5692, device='cuda:0')
[NeMo I 2022-02-01 21:15:48 wer:244] 
    
[NeMo I 2022-02-01 21:15:48 wer:245] reference:заканчивается еще один долгий но плодотворный рабочий день
[NeMo I 2022-02-01 21:15:48 wer:246] predicted:заканчиваейте ещё оиндолги наплахла на рабочудн
tensor(0.4324, device='cuda:0')
[NeMo I 2022-02-01 21:15:48 wer:244] 
    
[NeMo I 2022-02-01 21:15:48 wer:245] reference:давайте сосредоточ

In [ ]:
# На 15 эпохах
# WER = 0.5257873988896619
# 0.46

In [ ]:
# import IPython

audio_rus_file = ['/content/rus_text_audit_eto.mp3']

# IPython.display.Audio(audio_rus_file[0])

In [ ]:
#старая модель 
Our_quartznet4.transcribe(audio_rus_file)

[NeMo W 2022-01-24 02:50:59 audio_to_text_dataset:55] `labels` is explicitly provided to the data loader, and is different from the `labels` provided at the model level config.
    If this is incorrect, please set the dataloader's `labels` to None.


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['вмудренни адит это форма контрона деятельности организации изнутри  процетура помагает руководство промирять финансоввое состояния бизнаса и бестоявенрность отчетности']

In [ ]:
rus_from_eng.transcribe(audio_rus_file)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['внутренние алдит  это форма контроля деятельности организации изнутри процедура помогает руководство проверять финансоное состояние бизнеса и достоверности отчетности']

In [ ]:
# после 5ти эпохах
# внудрене алдить эта форма контроледительность организации изнутрипроцедура пумогает руководству привереть нанцакое состояния бизеса и бестоянность отчетности

# на 10ти эпохах 
# ввутреми алудит это форма контпралят деятельности организации изнутремпроцедуры омогает руководство провирательнарсовое состоянии безуса и достовирность обтерности

# на 15ти эпохах
# вмудренни адит это форма контрона деятельности организации изнутри  процетура помагает руководство промирять финансоввое состояния бизнаса и бестоявенрность отчетности

In [ ]:
entering(x_lat, y_lon, R)

0.0544348800890499


## Тест модели quartznet15x5 от сбера дообученного на самом большом размеченном в ручную датасете golos

In [5]:
!wget https://sc.link/ZMv

--2022-02-09 17:58:57--  https://sc.link/ZMv
Resolving sc.link (sc.link)... 45.89.227.39
Connecting to sc.link (sc.link)|45.89.227.39|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo [following]
--2022-02-09 17:59:00--  https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo
Resolving n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)... 37.18.122.129
Connecting to n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)|37.18.122.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71179174 (68M) [application/octet-stream]
Saving to: ‘ZMv’

ZMv                 100%[===================>]  67.88M  18.6MB/s    in 3.7s    

2022-02-09 17:59:04 (18.6 MB/s) - ‘ZMv’ saved [71179174/71179174]



In [6]:
sber_quartzNet = nemo_asr.models.EncDecCTCModel.restore_from("/content/ZMv")

[NeMo W 2022-02-09 17:59:05 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2022-02-09 17:59:05 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2022-02-09 17:59:05 features:255] PADDING: 16
[NeMo I 2022-02-09 17:59:05 features:272] STFT using torch
[NeMo I 2022-02-09 17:59:06 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-02-09 17:59:15 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /content/ZMv.


In [ ]:
dev_manifest

'manifests/ru/commonvoice_dev_manifest.json'

In [ ]:
sber_quartzNet._cfg['validation_ds']['manifest_filepath'] = dev_manifest
sber_quartzNet._cfg['validation_ds']

{'manifest_filepath': 'manifests/ru/commonvoice_dev_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 64, 'num_workers': 20, 'shuffle': False, 'parser': 'ru'}

In [ ]:
#либо баг, либо фитча внутри счберовской модели, там парсер с название 'ru', который модель не знает 
sber_quartzNet._cfg['validation_ds']['parser'] = 'en'

In [ ]:
sber_quartzNet._cfg['validation_ds']['batch_size'] = 14

#Настройка загрузчика тестовых данных
sber_quartzNet.setup_test_data(test_data_config=sber_quartzNet._cfg['validation_ds'])
sber_quartzNet.cuda()
sber_quartzNet.eval()

wer_nums = []
wer_denoms = []


for test_batch in sber_quartzNet.test_dataloader():
        print('='*20)
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    

        log_probs, encoded_len, greedy_predictions = sber_quartzNet(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        
        sber_quartzNet._wer.update(greedy_predictions, targets, targets_lengths)

        wer, wer_num, wer_denom= sber_quartzNet._wer.compute()
        print(wer)

        sber_quartzNet._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, targets, targets_lengths, encoded_len, greedy_predictions
        print('='*20)


# делим сумму всех числителей и знаменателей
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-01-23 21:41:55 collections:173] Dataset loaded with 7962 files totalling 12.36 hours
[NeMo I 2022-01-23 21:41:55 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-01-23 21:41:55 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


tensor(0.0208, device='cuda:0')
tensor(0.0730, device='cuda:0')
tensor(0.0552, device='cuda:0')
tensor(0.1026, device='cuda:0')
tensor(0.1488, device='cuda:0')
tensor(0.1515, device='cuda:0')
tensor(0.1732, device='cuda:0')
tensor(0.1081, device='cuda:0')
tensor(0.0752, device='cuda:0')
tensor(0.1500, device='cuda:0')
tensor(0.0462, device='cuda:0')
tensor(0.0221, device='cuda:0')
tensor(0.0229, device='cuda:0')
tensor(0.0714, device='cuda:0')
tensor(0.1127, device='cuda:0')
tensor(0.1314, device='cuda:0')
tensor(0.2292, device='cuda:0')
tensor(0.0368, device='cuda:0')
tensor(0.1028, device='cuda:0')
tensor(0.0726, device='cuda:0')
tensor(0.0559, device='cuda:0')
tensor(0.0882, device='cuda:0')
tensor(0.0563, device='cuda:0')
tensor(0.0451, device='cuda:0')
tensor(0.0696, device='cuda:0')
tensor(0.0946, device='cuda:0')
tensor(0.1280, device='cuda:0')
tensor(0.0698, device='cuda:0')
tensor(0.1301, device='cuda:0')
tensor(0.1250, device='cuda:0')
tensor(0.1448, device='cuda:0')
tensor(0

In [ ]:
!nvidia-smi

Sun Jan 23 21:38:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    33W / 250W |  15657MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Загрузка MCV для польского языка

In [8]:
!wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/get_commonvoice_data.py

--2022-02-09 17:59:17--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/get_commonvoice_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6778 (6.6K) [text/plain]
Saving to: ‘scripts/get_commonvoice_data.py’

get_commonvoice_dat 100%[===================>]   6.62K  --.-KB/s    in 0s      

2022-02-09 17:59:17 (97.4 MB/s) - ‘scripts/get_commonvoice_data.py’ saved [6778/6778]



In [9]:
!sudo apt-get install sox libsox-fmt-mp3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight

In [3]:
# необходимо скачать раздел MCV c польскими датасетами
VERSION = "cv-corpus-6.1-2020-12-11"
LANGUAGE = "pl"

tokenizer_dir = os.path.join('tokenizers', LANGUAGE)
manifest_dir = os.path.join('manifests', LANGUAGE)

In [4]:
manifest_dir

'manifests/pl'

In [10]:
!mkdir -p datasets

In [13]:
# !rm -R /content/datasets

In [14]:
!python scripts/get_commonvoice_data.py \
  --data_root "datasets/$LANGUAGE/" \
  --manifest_dir=$manifest_dir \
  --sample_rate=16000 \
  --n_channels=1 \
  --version=$VERSION \
  --language=$LANGUAGE \
  --files_to_process 'train.tsv' 'dev.tsv' 'test.tsv'

--2022-02-09 17:59:21--  https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/pl.tar.gz
Resolving voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)... 52.92.176.241
Connecting to voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)|52.92.176.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3537012341 (3.3G) [application/octet-stream]
Saving to: ‘datasets/pl/pl.tar.gz’

pl.tar.gz           100%[===================>]   3.29G  32.1MB/s    in 2m 2s   

2022-02-09 18:01:23 (27.7 MB/s) - ‘datasets/pl/pl.tar.gz’ saved [3537012341/3537012341]

100% 7467/7467 [03:05<00:00, 40.15it/s]
100% 7467/7467 [00:00<00:00, 94861.10it/s]
100% 5152/5152 [02:10<00:00, 39.46it/s]
100% 5152/5152 [00:00<00:00, 100224.73it

In [5]:
pl_train_manifest = f"{manifest_dir}/commonvoice_train_manifest.json"
pl_dev_manifest = f"{manifest_dir}/commonvoice_dev_manifest.json"
pl_test_manifest = f"{manifest_dir}/commonvoice_test_manifest.json"

## Поверка польской модели WER

In [30]:
#проверка обычной модели 
#загрузка ради алфавита
polish_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_pl_quartznet15x5")

[NeMo I 2022-02-09 19:26:18 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.7.0rc/stt_pl_quartznet15x5/a0688bd9116271719abfca130a32ac92/stt_pl_quartznet15x5.nemo.
[NeMo I 2022-02-09 19:26:18 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.7.0rc/stt_pl_quartznet15x5/a0688bd9116271719abfca130a32ac92/stt_pl_quartznet15x5.nemo
[NeMo I 2022-02-09 19:26:18 common:704] Instantiating model from pre-trained checkpoint


[NeMo W 2022-02-09 19:26:18 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - ą
    - b
    - c
    - ć
    - d
    - e
    - ę
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - ł
    - m
    - 'n'
    - ń
    - o
    - ó
    - p
    - r
    - s
    - ś
    - t
    - u
    - w
    - 'y'
    - z
    - ź
    - ż
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-02-09 19:26:18 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data l

[NeMo I 2022-02-09 19:26:18 features:255] PADDING: 16
[NeMo I 2022-02-09 19:26:18 features:272] STFT using torch
[NeMo I 2022-02-09 19:26:19 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-02-09 19:26:20 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.7.0rc/stt_pl_quartznet15x5/a0688bd9116271719abfca130a32ac92/stt_pl_quartznet15x5.nemo.


In [ ]:
#проверка wer польской модели 

In [ ]:
polish_model._cfg['validation_ds']['manifest_filepath'] = pl_dev_manifest
polish_model._cfg['validation_ds']

{'manifest_filepath': 'manifests/pl/commonvoice_dev_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż'], 'batch_size': 32, 'shuffle': False, 'num_workers': 8, 'pin_memory': True}

In [ ]:

polish_model._cfg['validation_ds']['batch_size'] = 14

#Настройка загрузчика тестовых данных
polish_model.setup_test_data(test_data_config=polish_model._cfg['validation_ds'])
polish_model.cuda()
polish_model.eval()

wer_nums = []
wer_denoms = []


for test_batch in polish_model.test_dataloader():
        print('='*20)
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    

        log_probs, encoded_len, greedy_predictions = polish_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        
        polish_model._wer.update(greedy_predictions, targets, targets_lengths)

        wer, wer_num, wer_denom= polish_model._wer.compute()
        print(wer)

        polish_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, targets, targets_lengths, encoded_len, greedy_predictions
        print('='*20)


# делим сумму всех числителей и знаменателей
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-01-24 04:28:36 audio_to_text_dataset:42] Model level config does not container `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2022-01-24 04:28:36 audio_to_text_dataset:42] Model level config does not container `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2022-01-24 04:28:36 collections:173] Dataset loaded with 5152 files totalling 6.70 hours
[NeMo I 2022-01-24 04:28:36 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-01-24 04:28:36 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


tensor(0.0104, device='cuda:0')
tensor(0.0294, device='cuda:0')
tensor(0.0208, device='cuda:0')
tensor(0.0545, device='cuda:0')
tensor(0.1207, device='cuda:0')
tensor(0.0762, device='cuda:0')
tensor(0.0241, device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0.1591, device='cuda:0')
tensor(0.0824, device='cuda:0')
tensor(0.1296, device='cuda:0')
tensor(0.0174, device='cuda:0')
tensor(0.2273, device='cuda:0')
tensor(0.0440, device='cuda:0')
tensor(0.0594, device='cuda:0')
tensor(0.0312, device='cuda:0')
tensor(0.0686, device='cuda:0')
tensor(0.1529, device='cuda:0')
tensor(0.0182, device='cuda:0')
tensor(0.2043, device='cuda:0')
tensor(0.0842, device='cuda:0')
tensor(0.0625, device='cuda:0')
tensor(0.1121, device='cuda:0')
tensor(0.0463, device='cuda:0')
tensor(0.1565, device='cuda:0')
tensor(0.0225, device='cuda:0')
tensor(0.0370, device='cuda:0')
tensor(0.0761, device='cuda:0')
tensor(0., device='cuda:0')
tensor(0.1935, device='cuda:0')
tensor(0.0761, devic

KeyboardInterrupt: ignored

In [ ]:
# wer=13%

## Дообучение русской модели от Сбера на польском языке

In [19]:
#ссылка на сберовскую
!wget https://sc.link/ZMv

--2022-02-09 12:57:11--  https://sc.link/ZMv
Resolving sc.link (sc.link)... 45.89.227.39
Connecting to sc.link (sc.link)|45.89.227.39|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo [following]
--2022-02-09 12:57:13--  https://n-ws-3jtx8.s3pd02.sbercloud.ru/b-ws-3jtx8-eir/golos/QuartzNet15x5_golos.nemo
Resolving n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)... 37.18.122.129
Connecting to n-ws-3jtx8.s3pd02.sbercloud.ru (n-ws-3jtx8.s3pd02.sbercloud.ru)|37.18.122.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71179174 (68M) [application/octet-stream]
Saving to: ‘ZMv’

ZMv                 100%[===================>]  67.88M  17.6MB/s    in 4.1s    

2022-02-09 12:57:19 (16.5 MB/s) - ‘ZMv’ saved [71179174/71179174]



In [122]:
polish_sber_quartz = nemo_asr.models.EncDecCTCModel.restore_from("/content/ZMv")

[NeMo W 2022-02-09 19:48:07 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2022-02-09 19:48:07 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2022-02-09 19:48:07 features:255] PADDING: 16
[NeMo I 2022-02-09 19:48:07 features:272] STFT using torch
[NeMo I 2022-02-09 19:48:08 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-02-09 19:48:08 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /content/ZMv.


In [94]:
!nvidia-smi

Wed Feb  9 19:41:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    35W / 250W |  10851MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## -------Изучения сходства и различий русской сберовской и англо==>польской обычной моделью из немо -------------------------
сберовской модели не хватает видеопамяти в колабе для дообучения

In [22]:
#польская модель (сберовская еще на русском языке)
polish_sber_quartz.cfg.keys()

dict_keys(['sample_rate', 'repeat', 'dropout', 'separable', 'labels', 'optim', 'train_ds', 'validation_ds', 'test_ds', 'preprocessor', 'spec_augment', 'encoder', 'decoder', 'target', 'nemo_version'])

In [23]:
 #русская (дообученная английская)
polish_model.cfg.keys()

dict_keys(['sample_rate', 'repeat', 'dropout', 'separable', 'labels', 'train_ds', 'validation_ds', 'preprocessor', 'spec_augment', 'encoder', 'decoder', 'optim', 'target', 'test_ds', 'nemo_version'])

In [95]:
list(rus_from_eng.cfg.keys())

['sample_rate',
 'repeat',
 'dropout',
 'separable',
 'labels',
 'train_ds',
 'validation_ds',
 'preprocessor',
 'spec_augment',
 'encoder',
 'decoder',
 'optim',
 'target',
 'test_ds',
 'nemo_version']

In [111]:
polish_sber_quartz.cfg['sample_rate']

16000

In [185]:
for key in rus_from_eng.cfg.keys():
  print(key, type(polish_sber_quartz.cfg[key]))

sample_rate <class 'int'>
repeat <class 'int'>
dropout <class 'float'>
separable <class 'bool'>
labels <class 'omegaconf.listconfig.ListConfig'>
train_ds <class 'omegaconf.dictconfig.DictConfig'>
validation_ds <class 'omegaconf.dictconfig.DictConfig'>
preprocessor <class 'omegaconf.dictconfig.DictConfig'>
spec_augment <class 'omegaconf.dictconfig.DictConfig'>
encoder <class 'omegaconf.dictconfig.DictConfig'>
decoder <class 'omegaconf.dictconfig.DictConfig'>
optim <class 'omegaconf.dictconfig.DictConfig'>
target <class 'str'>
test_ds <class 'omegaconf.dictconfig.DictConfig'>
nemo_version <class 'str'>


In [224]:
# for key in rus_from_eng.cfg.keys():
#   if str(type(polish_sber_quartz.cfg[key])) == "<class 'omegaconf.listconfig.ListConfig'>":
#     # print('f')
#   # and type(polish_sber_quartz.cfg[key]) != float and type(polish_sber_quartz.cfg[key]) != bool and type(polish_sber_quartz.cfg[key]) != list:
#     # print(key)
#     # try:
#     pd.DataFrame(polish_sber_quartz.cfg[key].items(), columns=['params', 'values_sber']).merge(pd.DataFrame(rus_from_eng.cfg[key].items(), columns=['params', 'values_nemo']), how='left', on='params')
#     # except 'ConfigAttributeError':
      # pass;/

In [42]:
#TRAIN
pd.DataFrame(polish_sber_quartz.cfg.train_ds.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.train_ds.items(), columns=['params', 'values_nemo']), how='left', on='params')

,params,values_sber,values_nemo
0,manifest_filepath,train/golos_and_mcv.jsonl,/raid/noneval.json
1,sample_rate,16000,16000
2,labels,"[ , а, б, в, г, д, е, ж, з, и, й, к, л, м, н, ...","[ , a, ą, b, c, ć, d, e, ę, f, g, h, i, j, k, ..."
3,batch_size,64,16
4,trim_silence,False,True
5,max_duration,20.0,16.7
6,min_duration,0.1,NaN
7,num_workers,20,8
8,shuffle,True,True
9,is_tarred,False,False


In [43]:
#TEST
# pd.DataFrame(polish_sber_quartz.cfg.test_ds.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.test_ds.items(), columns=['params', 'values_nemo']), how='left', on='params')

ConfigAttributeError: ignored

In [44]:
#VALID
pd.DataFrame(polish_sber_quartz.cfg.validation_ds.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.validation_ds.items(), columns=['params', 'values_nemo']), how='left', on='params')

,params,values_sber,values_nemo
0,manifest_filepath,"[test/mcv/dev_ru.jsonl, test/mcv/test_ru.jsonl...",/raid/dev.json
1,sample_rate,16000,16000
2,labels,"[ , а, б, в, г, д, е, ж, з, и, й, к, л, м, н, ...","[ , a, ą, b, c, ć, d, e, ę, f, g, h, i, j, k, ..."
3,batch_size,64,32
4,num_workers,20,8
5,shuffle,False,False
6,parser,ru,NaN


In [45]:
# preprocessor
pd.DataFrame(polish_sber_quartz.cfg.preprocessor.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.preprocessor.items(), columns=['params', 'values_nemo']), how='left', on='params')

,params,values_sber,values_nemo
0,_target_,nemo.collections.asr.modules.AudioToMelSpectro...,nemo.collections.asr.modules.AudioToMelSpectro...
1,normalize,per_feature,per_feature
2,window_size,0.02,0.02
3,sample_rate,16000,16000
4,window_stride,0.01,0.01
5,window,hann,hann
6,features,64,64
7,n_fft,512,512
8,frame_splicing,1,1
9,dither,0.00001,0.00001


In [46]:
#AUGMENT
pd.DataFrame(polish_sber_quartz.cfg.spec_augment.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.spec_augment.items(), columns=['params', 'values_nemo']), how='left', on='params')

,params,values_sber,values_nemo
0,_target_,nemo.collections.asr.modules.SpectrogramAugmen...,nemo.collections.asr.modules.SpectrogramAugmen...
1,rect_freq,50,50
2,rect_masks,5,5
3,rect_time,120,120


In [47]:
#ENCODER
pd.DataFrame( polish_sber_quartz.cfg.encoder.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.encoder.items(), columns=['params', 'values_nemo']), how='left', on='params')

,params,values_sber,values_nemo
0,_target_,nemo.collections.asr.modules.ConvASREncoder,nemo.collections.asr.modules.ConvASREncoder
1,feat_in,64,64
2,activation,relu,relu
3,conv_mask,True,True
4,jasper,"[[dilation, dropout, filters, kernel, repeat, ...","[[filters, repeat, kernel, stride, dilation, d..."


In [37]:
#DECODER
pd.DataFrame(polish_sber_quartz.cfg.decoder.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.decoder.items(), columns=['params', 'values_nemo']), how='left', on='params')

,params,values_sber,values_nemo
0,_target_,nemo.collections.asr.modules.ConvASRDecoder,nemo.collections.asr.modules.ConvASRDecoder
1,feat_in,1024,1024
2,num_classes,33,33
3,vocabulary,"[ , a, ą, b, c, ć, d, e, ę, f, g, h, i, j, k, ...","[ , a, ą, b, c, ć, d, e, ę, f, g, h, i, j, k, ..."


In [38]:
pd.DataFrame(polish_sber_quartz.cfg.optim.items(), columns=['params', 'values_sber']).merge(pd.DataFrame(polish_model.cfg.optim.items(), columns=['params', 'values_nemo']), how='left', on='params')

,params,values_sber,values_nemo
0,name,novograd,novograd
1,lr,0.0012,0.0012
2,betas,"[0.8, 0.5]","[0.8, 0.5]"
3,weight_decay,0.001,0.001
4,sched,"[name, warmup_steps, warmup_ratio, min_lr, las...","[name, warmup_steps, warmup_ratio, min_lr, las..."


In [113]:
polish_sber_quartz.cfg.labels, polish_model.cfg.labels

ConfigAttributeError: ignored

In [121]:
polish_sber_quartz.cfg.items()

[('sample_rate', 16000),
 ('repeat', 5),
 ('dropout', 0.0),
 ('separable', True),
 ('labels',
  [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']),
 ('optim',
  {'name': 'novograd', 'lr': 0.0012, 'betas': [0.8, 0.5], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': 500, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}),
 ('train_ds',
  {'manifest_filepath': 'manifests/pl/commonvoice_train_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż'], 'batch_size': 16, 'trim_silence': True, 'max_duration': 10.0, 'min_duration': 0.1, 'num_workers': 2, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None}),
 ('validation_ds',
  {'manifest_filepath': 'manifests/pl/commonvoice_te

In [120]:
polish_model.cfg.items()

[('preprocessor',
  {'_target_': 'nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor', 'normalize': 'per_feature', 'window_size': 0.02, 'sample_rate': 16000, 'window_stride': 0.01, 'window': 'hann', 'features': 64, 'n_fft': 512, 'frame_splicing': 1, 'dither': 1e-05, 'stft_conv': False}),
 ('spec_augment',
  {'_target_': 'nemo.collections.asr.modules.SpectrogramAugmentation', 'rect_freq': 50, 'rect_masks': 5, 'rect_time': 120}),
 ('encoder',
  {'_target_': 'nemo.collections.asr.modules.ConvASREncoder', 'feat_in': 64, 'activation': 'relu', 'conv_mask': True, 'jasper': [{'filters': 256, 'repeat': 1, 'kernel': [33], 'stride': [2], 'dilation': [1], 'dropout': 0.0, 'residual': False, 'separable': True}, {'filters': 256, 'repeat': 5, 'kernel': [33], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True}, {'filters': 256, 'repeat': 5, 'kernel': [33], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True}, {'filters': 256, 

In [39]:
p=[]
r=[]
other_par = ['sample_rate', 'repeat', 'dropout', 'separable']
for i in other_par:
  p.append(polish_sber_quartz.cfg[i])
  r.append(polish_model.cfg[i])
pd.DataFrame({'par': other_par, 'sber': p, 'nemo': r})


# polish_sber_quartz.cfg.sample_rate

NameError: ignored

## --------------------Продолжение-------------------------

In [123]:
#возьмем польский словарь 
# pol_voc = polish_model.cfg.decoder.vocabulary
pol_voc = [' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż']
len(pol_voc)

33

In [124]:
polish_sber_quartz.change_vocabulary(pol_voc)

[NeMo I 2022-02-09 19:48:19 ctc_models:365] Changed decoder to output to [' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż'] vocabulary.


In [125]:
#кол-во классов не меняется
polish_sber_quartz.cfg.decoder['num_classes'] #= len(pol_voc)

33

In [126]:
# проверка
polish_sber_quartz.cfg.decoder.vocabulary, len(polish_sber_quartz.cfg.decoder.vocabulary)

([' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż'],
 33)

In [127]:
polish_sber_quartz._cfg['train_ds']['manifest_filepath']  = pl_train_manifest
polish_sber_quartz._cfg['validation_ds']['manifest_filepath'] = pl_test_manifest

In [128]:
#вносим измненения по облегчению проццесса дообучения 
# 1. удаление парсера, которого в изначальной модели немо вроде нет
del polish_sber_quartz.cfg.train_ds.parser
del polish_sber_quartz.cfg.test_ds.parser
del polish_sber_quartz.cfg.validation_ds.parser

del  polish_sber_quartz.cfg.test_ds


In [130]:
# 2. измнение размеров батчей на меньшие
polish_sber_quartz.cfg.train_ds.batch_size = 16
# polish_sber_quartz.cfg.test_ds.batch_size = 8
polish_sber_quartz.cfg.validation_ds.batch_size = 32

In [131]:
# 3. вырезать места где ничего нет изменим на True, как в оригинальной немо модели
polish_sber_quartz.cfg.train_ds.trim_silence = True


In [132]:
# 4. изменим кол-во рабочих потоков на валидации на 1 
polish_sber_quartz.cfg.validation_ds.num_workers=2
polish_sber_quartz.cfg.train_ds.num_workers=2

In [133]:
# 5. изменим длину максимальной звуковой дорожки на меньшую
# 16
polish_sber_quartz.cfg.train_ds.max_duration=10.0

In [134]:
# 6. 
polish_sber_quartz.cfg.optim.betas = [0.8, 0.5]

In [135]:
# 7. 
del polish_sber_quartz.cfg.train_ds.tarred_shard_strategy

In [136]:
# 8.
polish_sber_quartz.cfg.optim.lr=0.0012

In [141]:
# странная хуйня которой нет у польской немо модели
polish_sber_quartz.cfg.labels = pol_voc

In [142]:
#ячейку нужно делать после всех преобразований и измнеений в настройках гиперпараметров внутри модели 

polish_sber_quartz.setup_training_data(train_data_config=polish_sber_quartz.cfg['train_ds'])
# sber_quartzNet._cfg['train_ds']['is_tarred'] = False
# polish_sber_quartz.setup_training_data(train_data_config=polish_sber_quartz._cfg['train_ds'])
polish_sber_quartz.setup_validation_data(val_data_config=polish_sber_quartz._cfg['validation_ds'])

[NeMo I 2022-02-09 19:50:51 collections:173] Dataset loaded with 7451 files totalling 9.29 hours
[NeMo I 2022-02-09 19:50:51 collections:174] 16 files were filtered totalling 0.05 hours
[NeMo I 2022-02-09 19:50:51 collections:173] Dataset loaded with 5152 files totalling 7.02 hours
[NeMo I 2022-02-09 19:50:51 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
# #не нужно менять парсер, если удалил его пунктом 2 ячейки выше
# polish_sber_quartz._cfg['validation_ds']['parser'] = 'en'
# polish_sber_quartz._cfg['train_ds']['parser'] = 'en'
# # polish_sber_quartz.cfg.test_ds.parser = 'en'

In [143]:
polish_sber_quartz.cfg.optim.sched, polish_model.cfg.optim.sched

{'name': 'CosineAnnealing', 'warmup_steps': 500, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}

### для работыа на TPU (лучше не трогать, хз как сделать, чтобы работало)

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html


     |████████████████████████████████| 149.9 MB 28 kB/s 
     |████████████████████████████████| 57 kB 2.6 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.10
    Uninstalling google-api-python-client-1.12.10:
      Successfully uninstalled google-api-python-client-1.12.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.295 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:02tcmalloc: large alloc 1147494400 bytes == 0x5580fb028000 @  0x7f1d617f4615 0x5580f834b4cc 0x5580f842b47a 0x5580f834e2ed 0x5580f843fe1d 0x5580f83c1e99 0x5580f83bc9ee 0x5580f834fbda 0x5580f83c1d00 0x558

In [ ]:
!pip install torch torchvision torchaudio pytorch-lightning
!pip install -r requirements.txt

     |████████████████████████████████| 527 kB 5.5 MB/s 
     |████████████████████████████████| 2.9 MB 43.3 MB/s 
     |████████████████████████████████| 2.9 MB 39.7 MB/s 
     |████████████████████████████████| 2.9 MB 41.2 MB/s 
     |████████████████████████████████| 1.9 MB 45.5 MB/s 
     |████████████████████████████████| 1.9 MB 54.5 MB/s 
     |████████████████████████████████| 396 kB 68.9 MB/s 
     |████████████████████████████████| 133 kB 32.6 MB/s 
     |████████████████████████████████| 829 kB 46.2 MB/s 
     |████████████████████████████████| 952 kB 54.2 MB/s 
     |████████████████████████████████| 596 kB 57.4 MB/s 
     |████████████████████████████████| 1.1 MB 62.1 MB/s 
     |████████████████████████████████| 144 kB 51.9 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 44.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=4329a1b61e96f3114bf5fbb5769501e8e6f5629096ac9d

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
trainer = pl.Trainer(tpu_cores=[8], max_epochs=10)

GPU available: False, used: False
TPU available: True, using: [8] TPU cores
IPU available: False, using: 0 IPUs


## GPU или CPU

In [145]:
import pytorch_lightning as pl
#прописываем кол-во гпу и эпох 
# trainer = pl.Trainer(gpus=1, max_epochs=10)

In [63]:
#заморозка слоев энкодера

# import torch
# import torch.nn as nn

# def enable_bn_se(m):
#     if type(m) == nn.BatchNorm1d:
#         m.train()
#         for param in m.parameters():
#             param.requires_grad_(True)

#     if 'SqueezeExcite' in type(m).__name__:
#         m.train()
#         for param in m.parameters():
#             param.requires_grad_(True)

# polish_sber_quartz.encoder.freeze()
# polish_sber_quartz.encoder.apply(enable_bn_se)

In [180]:
#если не прописывать явно gpus, то модель считается на cpu
trainer = pl.Trainer(gpus=1, max_epochs=50)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [147]:
#чтобы смотреть логи во время обучения, показывают референс и предикт
polish_sber_quartz._wer.log_prediction = True

In [174]:
!nvidia-smi

Wed Feb  9 21:08:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    35W / 250W |   4931MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [187]:
                      
polish_sber_quartz.cfg.optim.lr = 0.001

In [188]:
trainer.fit(polish_sber_quartz)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2022-02-10 00:36:41 modelPT:470] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-02-10 00:36:41 modelPT:582] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2022-02-10 00:36:41 lr_scheduler:817] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | _wer              | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 34.9 K
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.718    Total estimated model params size (MB)
[NeMo W 2022-02-10 00:36:42 nemo_logging:349] /usr/loc

Validation sanity check: 0it [00:00, ?it/s]

[NeMo I 2022-02-10 00:36:42 wer:244] 
    
[NeMo I 2022-02-10 00:36:42 wer:245] reference:gdzież tu powód do wyrzutów
[NeMo I 2022-02-10 00:36:42 wer:246] predicted:gdzieżz tu powód doworzył dów
[NeMo I 2022-02-10 00:36:42 wer:244] 
    
[NeMo I 2022-02-10 00:36:43 wer:245] reference:dziwożona łasa jest na miód i pszczołom go stale wyjada
[NeMo I 2022-02-10 00:36:43 wer:246] predicted:dziwożona łasa jeznamiuót i przczołąm go stale wyjada


...

Validating: 0it [00:00, ?it/s]

[NeMo I 2022-02-10 00:59:58 wer:244] 
    
[NeMo I 2022-02-10 00:59:58 wer:245] reference:gdzież tu powód do wyrzutów
[NeMo I 2022-02-10 00:59:58 wer:246] predicted: gdzies to powód doworydów
[NeMo I 2022-02-10 00:59:59 wer:244] 
    
[NeMo I 2022-02-10 00:59:59 wer:245] reference:dziwożona łasa jest na miód i pszczołom go stale wyjada
[NeMo I 2022-02-10 00:59:59 wer:246] predicted:dziłożona łasa je znamiut i przczołąm go stale wyjada
[NeMo I 2022-02-10 00:59:59 wer:244] 
    
[NeMo I 2022-02-10 00:59:59 wer:245] reference:uznałem iż stanowisko to właśnie mi odpowiada
[NeMo I 2022-02-10 00:59:59 wer:246] predicted:znałem i stanowisko to właśnie mi odpowiada
[NeMo I 2022-02-10 00:59:59 wer:244] 
    
[NeMo I 2022-02-10 00:59:59 wer:245] reference:poza saniami biegło trzydziestu ludzi otulonych w filtra
[NeMo I 2022-02-10 00:59:59 wer:246] predicted:pozasaniami jiegło przyjesgigłom sildra
[NeMo I 2022-02-10 01:00:00 wer:244] 
    
[NeMo I 2022-02-10 01:00:00 wer:245] reference:łaska pańs

Validating: 0it [00:00, ?it/s]

[NeMo I 2022-02-10 01:04:48 wer:244] 
    
[NeMo I 2022-02-10 01:04:48 wer:245] reference:gdzież tu powód do wyrzutów
[NeMo I 2022-02-10 01:04:48 wer:246] predicted: gdzież tu powód dowożydów
[NeMo I 2022-02-10 01:04:49 wer:244] 
    
[NeMo I 2022-02-10 01:04:49 wer:245] reference:dziwożona łasa jest na miód i pszczołom go stale wyjada
[NeMo I 2022-02-10 01:04:49 wer:246] predicted:dziłożona własa jest namiut i przczołąm go stale wyjada
[NeMo I 2022-02-10 01:04:49 wer:244] 
    
[NeMo I 2022-02-10 01:04:49 wer:245] reference:uznałem iż stanowisko to właśnie mi odpowiada
[NeMo I 2022-02-10 01:04:49 wer:246] predicted:pznałem i stanowisko to właśnie mi odpowiada
[NeMo I 2022-02-10 01:04:49 wer:244] 
    
[NeMo I 2022-02-10 01:04:49 wer:245] reference:poza saniami biegło trzydziestu ludzi otulonych w filtra
[NeMo I 2022-02-10 01:04:49 wer:246] predicted:pozasaniami wiegło przydziesdzi yłom silcera
[NeMo I 2022-02-10 01:04:50 wer:244] 
    
[NeMo I 2022-02-10 01:04:50 wer:245] reference:ła

[NeMo W 2022-02-10 01:05:52 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    


In [189]:
 #Сохранение

save_path = f"pol_from_rus_sber_quartznet50_epoch-{LANGUAGE}.nemo"
polish_sber_quartz.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")


Model saved at path : /content/pol_from_rus_sber_quartznet50_epoch-pl.nemo


In [190]:
#Загрузка раннее дообученной польской модели
polish_from_sberrus = nemo_asr.models.EncDecCTCModel.restore_from("/content/pol_from_rus_sber_quartznet50_epoch-pl.nemo")

[NeMo W 2022-02-10 01:06:16 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: manifests/pl/commonvoice_train_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - ą
    - b
    - c
    - ć
    - d
    - e
    - ę
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - ł
    - m
    - 'n'
    - ń
    - o
    - ó
    - p
    - r
    - s
    - ś
    - t
    - u
    - w
    - 'y'
    - z
    - ź
    - ż
    batch_size: 16
    trim_silence: true
    max_duration: 10.0
    min_duration: 0.1
    num_workers: 2
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-02-10 01:06:16 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to 

[NeMo I 2022-02-10 01:06:16 features:255] PADDING: 16
[NeMo I 2022-02-10 01:06:16 features:272] STFT using torch
[NeMo I 2022-02-10 01:06:16 audio_preprocessing:494] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-02-10 01:06:17 save_restore_connector:154] Model EncDecCTCModel was successfully restored from /content/pol_from_rus_sber_quartznet50_epoch-pl.nemo.


In [184]:
polish_sber_quartz._cfg['validation_ds']

{'manifest_filepath': 'manifests/pl/commonvoice_test_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż'], 'batch_size': 32, 'num_workers': 2, 'shuffle': False}

In [191]:
#Проверка WER
polish_from_sberrus._cfg['validation_ds']['manifest_filepath'] = pl_dev_manifest
polish_from_sberrus._cfg['validation_ds']

{'manifest_filepath': 'manifests/pl/commonvoice_dev_manifest.json', 'sample_rate': 16000, 'labels': [' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż'], 'batch_size': 32, 'num_workers': 2, 'shuffle': False}

In [192]:

polish_from_sberrus._cfg['validation_ds']['batch_size'] = 8

#Настройка загрузчика тестовых данных
polish_from_sberrus.setup_test_data(test_data_config=polish_from_sberrus._cfg['validation_ds'])
polish_from_sberrus.cuda()
polish_from_sberrus.eval()

wer_nums = []
wer_denoms = []


for test_batch in polish_from_sberrus.test_dataloader():
        print('='*20)
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]    

        log_probs, encoded_len, greedy_predictions = polish_from_sberrus(
            input_signal=test_batch[0], input_signal_length=test_batch[1])
        
        polish_from_sberrus._wer.update(greedy_predictions, targets, targets_lengths)

        wer, wer_num, wer_denom= polish_from_sberrus._wer.compute()
        print(wer)

        polish_from_sberrus._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, targets, targets_lengths, encoded_len, greedy_predictions
        print('='*20)


# делим сумму всех числителей и знаменателей
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2022-02-10 01:06:22 collections:173] Dataset loaded with 5152 files totalling 6.70 hours
[NeMo I 2022-02-10 01:06:22 collections:174] 0 files were filtered totalling 0.00 hours
tensor(0.5246, device='cuda:0')
tensor(0.4118, device='cuda:0')
tensor(0.4355, device='cuda:0')
tensor(0.3667, device='cuda:0')
tensor(0.2963, device='cuda:0')
tensor(0.4545, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.3857, device='cuda:0')
tensor(0.5818, device='cuda:0')
tensor(0.4386, device='cuda:0')
tensor(0.4333, device='cuda:0')
tensor(0.5294, device='cuda:0')
tensor(0.3824, device='cuda:0')
tensor(0.1429, device='cuda:0')
tensor(0.2714, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.3725, device='cuda:0')
tensor(0.4364, device='cuda:0')
tensor(0.4146, device='cuda:0')
tensor(0.5507, device='cuda:0')
tensor(0.3966, device='cuda:0')
tensor(0.3676, device='cuda:0')
tensor(0.3548, device='cuda:0')
tensor(0.6735, device='cuda:0')
tensor(0.4082, device='cuda:0')
tensor(0.4423, 